In [1]:
%pylab inline
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
import tensorflow_datasets as tfds
from outlier.datasets import gaussian_convergence

In [3]:
data_set, info = tfds.load(name="gaussian_convergence", split="train", data_dir='/global/cscratch1/sd/vboehm/Datasets/Gaussian_Kappa',with_info=True)

In [4]:
DATASET_SIZE = info.splits['train'].num_examples

In [5]:
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

full_dataset = data_set
full_dataset = full_dataset.shuffle(buffer_size=DATASET_SIZE)
train_dataset= full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset  = test_dataset.skip(test_size)
test_dataset = test_dataset.take(test_size)

print(train_dataset)

<DatasetV1Adapter shapes: {map: (256, 256), params: (2,)}, types: {map: tf.float32, params: tf.float32}>


In [6]:
def whiten(data):
    x    = data['map'] 
    mean = tf.reduce_mean(x)
    std  = tf.math.reduce_std(x)
    x    = (x-mean)/std
    x    = tf.expand_dims(x,-1)
    return x, data['params']

def downsample(x,y):
    x    = tf.image.resize(x,[128,128])
    return x, y

In [7]:
# for example in train_dataset.take(1):  # Only take a single example
#     kappa_map, params = example["map"], example["params"]
#     imshow(np.squeeze(kappa_map))
#     print(params)
#     colorbar()

In [8]:
train_dataset = train_dataset.batch(32).shuffle(1000)
train_dataset = train_dataset.map(whiten)
train_dataset = train_dataset.map(downsample)
train_dataset = train_dataset.repeat()
val_dataset = val_dataset.batch(64).shuffle(1000)
val_dataset = val_dataset.map(whiten)
val_dataset = val_dataset.map(downsample)
val_dataset = val_dataset.repeat()

In [9]:
val_dataset

<DatasetV1Adapter shapes: ((None, 128, 128, 1), (None, 2)), types: (tf.float32, tf.float32)>

In [10]:
def res_net_block(input_data, filters, conv_size):
    x = layers.Conv2D(filters, conv_size, activation='relu', padding='same')(input_data)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters, conv_size, activation=None, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, input_data])
    x = layers.Activation('relu')(x)
    return x

In [11]:
def non_res_block(input_data, filters, conv_size):
    x = layers.Conv2D(filters, conv_size, activation='relu', padding='same')(input_data)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters, conv_size, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    return x

In [12]:
inputs = keras.Input(shape=(128, 128, 1))
x = layers.Conv2D(32, 1, activation='relu')(inputs)
x = layers.Conv2D(64, 1, activation='relu')(x)
x = layers.MaxPooling2D(1)(x)
num_res_net_blocks = 3
for i in range(num_res_net_blocks):
    x = res_net_block(x, 64, 1)
x = layers.Conv2D(64, 1, activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(2)(x)

res_net_model = keras.Model(inputs, outputs)

In [13]:
res_net_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 32) 64          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 64) 2112        conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [ ]:
# callbacks = [
# # Write TensorBoard logs to `./logs` directory
#     keras.callbacks.TensorBoard(log_dir='./log/{}'.format(dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")), write_images=True),
# ]
res_net_model.compile(optimizer=keras.optimizers.Adam(),loss='mse',metrics=['mse'])
res_net_model.fit(train_dataset, epochs=3, steps_per_epoch=200,
              validation_data=val_dataset,verbose=1,
              validation_steps=3)#, callbacks=callbacks)

Train for 200 steps, validate for 3 steps
Epoch 1/3
 29/200 [===>..........................] - ETA: 15:36 - loss: 0.1339 - mse: 0.1339